<hr style="height:3px;border:none;color:#333;background-color:#333;" />
<img style=" float:right; display:inline" src=https://upload.wikimedia.org/wikipedia/en/7/74/TensorFlow.png width="150" height="150" />

### ** First Model in TensorFlow ** 
<br/>
<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 2.0em;"> **Hands on Tutorial on TensorFlow** </span>

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 2em;"> **Outline** </span>  
<br/>
- <span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.2em;"> Clues in TensorFlow </span> 
- <span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.4em;"> ** First Model in TensorFlow ** </span> 
- <span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.2em;"> Convolutional Neural Network (CNN) on Cifar-10
 </span>  
- <span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.2em;"> Techniques for Deep Learning in TensorFlow </span>
- <span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.2em;"> Inception V3 </span>
- <span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.2em;"> CNN for Text Classification </span>
- <span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.2em;"> Hidden Markov Model in TensorFlow </span>
- <span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.2em;"> Image Captioning in TensorFlow </span>
<br/>
<br/>

This tutorial is an explanation, line by line, of how you can implement the two layer CNN on the MNIST dataset.

#### At the end of this tutorial you learn:
- Simple Code of implementing CNN in TensorFlow, You can expand it for your project
- The clue things in ML
- Transfering Parametrs between Layers


#### The MNIST Dataset [1]:
The MNIST database of handwritten digits, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.
It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

Four files are available on this site:

train-images-idx3-ubyte.gz:  training set images (9912422 bytes)  
train-labels-idx1-ubyte.gz:  training set labels (28881 bytes)  
t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes)  
t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes)  
  
[1] http://yann.lecun.com/exdb/mnist/

#### The Model
The Model we implement is as follow:

![Image of Yaktocat](pic/CNN.png)

![Image of Yaktocat](pic/Model.png)

### Code

In [1]:
import tensorflow as tf

For the ease of access this inpute.data provides you with the requried trainig data, trainig lable, test data, and test lable

In [2]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz


/usr/lib/python2.7/gzip.py:268: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:55: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

In [4]:
x_image = tf.reshape(x, [-1,28,28,1])
print "x_image="
print x_image

x_image=
Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)


In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [6]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])



h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [7]:
sess = tf.Session()

sess.run(tf.initialize_all_variables())
for i in range(200):
  batch = mnist.train.next_batch(50)
  if i%10 == 0:
        train_accuracy = sess.run( accuracy, feed_dict={
                x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

  sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"% sess.run(accuracy, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 10, training accuracy 0.16
step 20, training accuracy 0.34
step 30, training accuracy 0.56
step 40, training accuracy 0.54
step 50, training accuracy 0.6
step 60, training accuracy 0.74
step 70, training accuracy 0.86
step 80, training accuracy 0.86
step 90, training accuracy 0.76
step 100, training accuracy 0.82
step 110, training accuracy 0.88
step 120, training accuracy 0.9
step 130, training accuracy 0.84
step 140, training accuracy 0.9
step 150, training accuracy 0.82
step 160, training accuracy 0.8
step 170, training accuracy 0.84
step 180, training accuracy 0.88
step 190, training accuracy 0.78
test accuracy 0.9038
